In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor as RandomForest
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.cross_validation import train_test_split

In [10]:
from __future__ import print_function
from rdkit import Chem

In [37]:
from sklearn.ensemble import GradientBoostingRegressor as GradientBoost
from sklearn.grid_search import GridSearchCV

In [12]:
from rdkit.Chem import AllChem


In [16]:
test = Chem.MolFromSmiles(df_train.ix[1,0])

NameError: name 'df_train' is not defined

In [17]:
"""
Read in train and test a?s Pandas DataFrames
"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [15]:
AllChem.Compute2DCoords(test)
print(Chem.MolToMolBlock(test))

NameError: name 'test' is not defined

In [3]:
df_train.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256,gap
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.19
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.60
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.49
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.36
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.98


In [5]:
df_train.shape

(1000000, 258)

In [4]:
df_test.head()

,Id,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,1,c1sc(-c2cnc3c(c2)c2nsnc2c2cc4cccnc4cc32)c2cc[n...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,[nH]1cccc1-c1cc2c3nsnc3c3c4sccc4[nH]c3c2s1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,[nH]1c2cc(-c3ccc[se]3)c3nsnc3c2c2c3cscc3c3ccc4...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,[nH]1c(cc2cnc3c(c12)c1=C[SiH2]C=c1c1ccc2=CCC=c...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,c1sc(-c2sc(-c3sc(-c4scc5[se]ccc45)c4ccoc34)c3c...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
#store gap values
Y_train = df_train.gap.values
#row where testing examples start
test_idx = df_train.shape[0]
#delete 'Id' column
df_test = df_test.drop(['Id'], axis=1)
#delete 'gap' column
df_train = df_train.drop(['gap'], axis=1)

In [19]:
#DataFrame with all train and test examples so we can more easily apply feature engineering on
df_all = pd.concat((df_train, df_test), axis=0)
df_all.head()

,smiles,feat_001,feat_002,feat_003,feat_004,feat_005,feat_006,feat_007,feat_008,feat_009,...,feat_247,feat_248,feat_249,feat_250,feat_251,feat_252,feat_253,feat_254,feat_255,feat_256
0,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
"""
Example Feature Engineering

this calculates the length of each smile string and adds a feature column with those lengths
Note: this is NOT a good feature and will result in a lower score!
"""
#smiles_len = np.vstack(df_all.smiles.astype(str).apply(lambda x: len(x)))
#df_all['smiles_len'] = pd.DataFrame(smiles_len)


'\nExample Feature Engineering\n\nthis calculates the length of each smile string and adds a feature column with those lengths\nNote: this is NOT a good feature and will result in a lower score!\n'

In [22]:
#Drop the 'smiles' column
df_all = df_all.drop(['smiles'], axis=1)
vals = df_all.values
X_train = vals[:test_idx]
X_test = vals[test_idx:]
#print "Train features:", X_train.shape
#print "Train gap:", Y_train.shape
#print "Test features:", X_test.shape

In [28]:
Y_train[0:5]

array([ 1.19,  1.6 ,  1.49,  1.36,  1.98])

In [38]:
LR = LinearRegression()
LR.fit(X_train, Y_train)
LR_pred = LR.predict(X_test)

In [47]:
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [ ]:
#Changed parameters
RF = RandomForestRegressor()
RF.fit(X_train, Y_train)
RF_pred = RF.predict(X_test)

In [16]:
# Parameters for tuning
depths = np.arange(2, 10)  # Trees and depth are explored on an exponentially growing space,
min_sample_leaf = np.arange(10,90, 20)  
#max_features = "sqrt"
# To keep track of the best model
best_score = 0
best_depth = 0
best_min_sample = 0 

# Run grid search for model with 5-fold cross validation
for depth in depths:
    for min_sample in min_sample_leaf:
        # Cross validation for every experiment
        k_folds = KFold(len(X_train[:,1]),n_folds=5, shuffle=True)
        scores = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = X_train[train_indices]
            y_train_cv = Y_train[train_indices]
            # Generate validation data
            x_validate =  X_train[validation_indices]
            y_validate = Y_train[validation_indices]

            # Fit random forest on training data
            model = RandomForestRegressor(n_estimators=64, max_depth=depth,min_samples_leaf=min_sample,max_features="sqrt")
            model.fit(x_train_cv, y_train_cv)
            # Score using accuracy of validation set
            scores.append(model.score((x_validate), y_validate))
            # Record and report accuracy
            average_score = np.mean(scores)

            # Update our record of the best parameters see so far
            if average_score > best_score:
                best_score = average_score
                best_depth = depth
                best_min_sample = min_sample

In [20]:
print(best_depth, best_score, best_min_sample)

(9, 0.42059617261025806, 50)


In [29]:
best_model = RandomForestRegressor(n_estimators=64, max_depth=9,min_samples_leaf=50,max_features="sqrt")
best_model.fit(X_train, Y_train)
Y_pred=best_model.predict(X_test)

In [53]:
len(X_train)

1000000

In [31]:
X_train_2,X_val_2,Y_train_2,Y_val_2 = train_test_split(X_train, Y_train,test_size=0.2)

array([ 1.6 ,  2.19,  2.1 , ...,  1.93,  1.11,  1.97])

In [33]:
#Random Forest try 2
min_samples_leaf=[10,50,250,500,1000, 5000, 10000,50000]
best_MSE = 9223372036854775807
best_min_sample = 0

#best_depth = 0
#best_min_sample = 0 
for min_sample in min_samples_leaf :
    model = RandomForest(n_estimators=200, oob_score = True, n_jobs = -1,random_state =50,max_features = "auto", min_samples_leaf=min_sample)
    model.fit(X_train_2, Y_train_2)
    Y_pred = model.predict(X_val_2)
    a = MSE(Y_val_2,Y_pred)
    if a < best_MSE:
                best_MSE= a
                best_min_sample = min_sample

In [36]:
print("The best min samples is: ",best_min_sample,"The best MSE is: ", best_MSE)

The best min samples is:  10 The best MSE is:  0.0744450644117


In [24]:
best_model2 = RandomForest(n_estimators=200, max_depth=9,oob_score = True, n_jobs = -1,random_state =50, min_samples_leaf=1000)
best_model2.fit(X_train, Y_train)
Y_pred2=best_model2.predict(X_test)

In [29]:
best_model2 = RandomForest(n_estimators=200, oob_score = True, n_jobs = -1,random_state =50, min_samples_leaf=1000)
best_model2.fit(X_train, Y_train)
Y_pred2=best_model2.predict(X_test)

In [ ]:
# Gradient Boosting  


##First tune max_depth and num_samples_split

min_samples_split = 0
min_samples_leaf = 0
min_weight_fraction_leaf = 0 
max_depth = 0 
max_leaf_nodes = 0 
max_features = 0 
# Run grid search for model with 5-fold cross validation
for depth in depths:
    for min_sample in min_sample_leaf:
        # Cross validation for every experiment
        k_folds = KFold(len(X_train[:,1]),n_folds=5, shuffle=True)
        scores = []
        for train_indices, validation_indices in k_folds:
            # Generate training data
            x_train_cv = X_train[train_indices]
            y_train_cv = Y_train[train_indices]
            # Generate validation data
            x_validate =  X_train[validation_indices]
            y_validate = Y_train[validation_indices]

            # Fit random forest on training data
            model = GradientBoost(loss="ls",learning_rate=)
            model.fit(x_train_cv, y_train_cv)
            # Score using accuracy of validation set
            scores.append(model.score((x_validate), y_validate))
            # Record and report accuracy
            average_score = np.mean(scores)

            # Update our record of the best parameters see so far
            if average_score > best_score:
                best_score = average_score
                best_depth = depth
                best_min_sample = min_sample

In [26]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [27]:
write_to_file("test3rf.csv", Y_pred2)